In [ ]:
# Cell 1: Install Required Packages

# Install core ML, NLP, and data handling packages silently
!pip install -q scikit-learn pandas joblib numpy nltk spacy transformers torch emoji streamlit pyngrok

# Download SpaCy English model (small) for NER
!python -m spacy download en_core_web_sm

# Download NLTK resources for Twitter dataset & text processing
import nltk
nltk.download('stopwords')       # Stopwords for text cleaning
nltk.download('punkt')           # Tokenizer
nltk.download('twitter_samples') # Sample tweets dataset

#Explanation:
#Installs all necessary packages for ML, NLP, and deployment.
#Downloads SpaCy model en_core_web_sm for named entity recognition.
#Downloads NLTK datasets for sentiment analysis and text cleaning.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 63.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [ ]:
# Cell 2: Import Required Libraries

# Data Handling
import pandas as pd
import numpy as np
import joblib
import random
import re

# NLP
from nltk.corpus import twitter_samples, stopwords
from nltk.tokenize import TweetTokenizer
import spacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# ML Models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Streamlit & Deployment
import streamlit as st
from pyngrok import ngrok

# Global Settings
RANDOM_STATE = 42  # Ensures reproducibility

#Explanation:
#Organized imports into categories: data handling, NLP, ML, deployment.
#RANDOM_STATE is defined globally for reproducibility.

In [ ]:
# Cell 3: Text Cleaning Function

# Initialize TweetTokenizer for tokenizing tweets
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stop_words = set(stopwords.words('english'))  # English stopwords

def clean_text(text):
    """
    Cleans input text by:
    1. Removing URLs and mentions
    2. Tokenizing using TweetTokenizer
    3. Removing stopwords
    4. Returning cleaned string
    """
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)            # Remove @mentions
    tokens = tokenizer.tokenize(text)           # Tokenize text
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]  # Remove stopwords & non-alpha
    return " ".join(tokens)

#Explanation:
#Central text cleaning function used for all datasets.
#Removes noise from text like URLs, mentions, and stopwords.
#Keeps only alphabetical words.

In [ ]:
# Cell 4: Sentiment Dataset (NLTK Twitter)

# Load positive and negative tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

# Create DataFrame
df_sentiment = pd.DataFrame({
    'text': positive_tweets + negative_tweets,
    'sentiment': ['positive']*len(positive_tweets) + ['negative']*len(negative_tweets)
})

# Clean text
df_sentiment['clean_text'] = df_sentiment['text'].apply(clean_text)

# Shuffle and reset index for randomness
df_sentiment = df_sentiment.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Quick check
print("Sentiment dataset size:", len(df_sentiment))
print(df_sentiment['sentiment'].value_counts())
df_sentiment.head()

#Explanation:
#Loads NLTK Twitter dataset for sentiment analysis.
#Applies clean_text() for preprocessing.
#Shuffles dataset to remove ordering bias.


Sentiment dataset size: 10000
sentiment
negative    5000
positive    5000
Name: count, dtype: int64


,text,sentiment,clean_text
0,"I love you, how but you? @Taecyeon2pm8 did you...",negative,love feel emm think
1,@mayusushita @dildeewana_ @sonalp2591 @deepti_...,positive,thanks guys
2,"Your love, O Lord, is better than life. :) &lt...",positive,love lord better life
3,@yasminyasir96 yeah but it will be better if w...,positive,yeah better use official account like
4,Ok good night I wish troye wasn't ugly and I m...,positive,ok good night wish troye ugly met today ok tod...


In [ ]:
# Cell 5: Intent Dataset

# Define small intent dataset
data_intent = {
    "text": [
        "hello there","hi buddy","good morning","ok bye","see you soon",
        "what time is the meeting","can you help me","i feel sad today",
        "great job team","i reached the office","the package is on the table"
    ],
    "intent": [
        "greeting","greeting","greeting","goodbye","goodbye",
        "question","question","complaint","praise","info","info"
]
}

df_intent = pd.DataFrame(data_intent)
df_intent['clean_text'] = df_intent['text'].apply(clean_text)

# Quick check
print("Intent dataset size:", len(df_intent))
print(df_intent['intent'].value_counts())
df_intent.head()

#Explanation:
#Small custom dataset for intent classification.
#Applies the same cleaning function for consistency.


Intent dataset size: 11
intent
greeting     3
goodbye      2
question     2
info         2
complaint    1
praise       1
Name: count, dtype: int64


,text,intent,clean_text
0,hello there,greeting,hello
1,hi buddy,greeting,hi buddy
2,good morning,greeting,good morning
3,ok bye,goodbye,ok bye
4,see you soon,goodbye,see soon


In [ ]:
# Cell 6: Large Synthetic Intent Dataset (df_big)

# Define phrases per intent
phrases_dict = {
    "greeting": ["hello there", "hi friend", "hey buddy", "good morning", "good evening", "hello how are you", "hi what's up", "hey there"],
    "goodbye": ["bye take care", "see you soon", "talk later", "good night", "i have to go now", "catch you later", "see you tomorrow"],
    "question": ["can you help me", "what time is the meeting", "how does this work", "where can i find it", "is anyone available", "can i ask something", "what should i do", "where do i submit"],
    "complaint": ["i feel tired and low today", "this app is frustrating", "my order arrived broken", "i am not happy with this", "this is really disappointing", "everything is going wrong", "i am feeling stressed", "the service is terrible", "i hate this situation"],
    "praise": ["thanks so much", "great job team", "i love this feature", "this is awesome", "amazing experience", "you did wonderful work", "nice work", "really helpful"],
    "info": ["i reached the office", "the delivery came today", "i will join soon", "i am at home now", "package is on the table", "meeting has started", "i am ready for the call"]
}

# Generate 300 examples per intent with minor punctuation variations
dataset = []
for intent, phrases in phrases_dict.items():
    for _ in range(300):
        text = random.choice(phrases) + " " + random.choice(["", "!", ".", "??", "!!"])
        dataset.append([text, intent])

# Convert to DataFrame
df_big = pd.DataFrame(dataset, columns=["text", "intent"])
df_big['clean_text'] = df_big['text'].apply(clean_text)

print("Large intent dataset (df_big) created. Total examples:", len(df_big))
df_big['intent'].value_counts()

#Explanation:
#Expands intent dataset to large synthetic dataset for better model training.
#Introduces minor punctuation variations to simulate real user inputs.


Large intent dataset (df_big) created. Total examples: 1800


,count
intent,
greeting,300
goodbye,300
question,300
complaint,300
praise,300
info,300


In [ ]:
# Cell 7: Train Sentiment Classifier

# Split features and labels
X_sent = df_sentiment['clean_text'].values
y_sent = df_sentiment['sentiment'].values

# Train-test split
X_train_sent, X_test_sent, y_train_sent, y_test_sent = train_test_split(
    X_sent, y_sent, test_size=0.2, random_state=RANDOM_STATE, stratify=y_sent
)

# TF-IDF vectorization (unigrams + bigrams)
tfidf_sent = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.95)
X_train_sent_tfidf = tfidf_sent.fit_transform(X_train_sent)
X_test_sent_tfidf = tfidf_sent.transform(X_test_sent)

# Logistic Regression classifier
clf_sent = LogisticRegression(max_iter=200, random_state=RANDOM_STATE)
clf_sent.fit(X_train_sent_tfidf, y_train_sent)

# Evaluate
y_pred_sent = clf_sent.predict(X_test_sent_tfidf)
acc_sent = accuracy_score(y_test_sent, y_pred_sent)
print(f"Sentiment Accuracy: {acc_sent:.4f}")
print(classification_report(y_test_sent, y_pred_sent, digits=4))

# Save model and vectorizer
joblib.dump(tfidf_sent, 'tfidf_sent.joblib')
joblib.dump(clf_sent, 'clf_sent.joblib')

#Explanation:
#TF-IDF vectorization converts text to numeric features.
#Logistic Regression trained on sentiment data.
#Evaluates model and saves for Streamlit usage.


Sentiment Accuracy: 0.7505
              precision    recall  f1-score   support

    negative     0.7335    0.7870    0.7593      1000
    positive     0.7702    0.7140    0.7410      1000

    accuracy                         0.7505      2000
   macro avg     0.7518    0.7505    0.7502      2000
weighted avg     0.7518    0.7505    0.7502      2000



['clf_sent.joblib']

In [ ]:
# Cell 8: Train Intent Classifier (df_big)

# Features and labels
X_intent = df_big['clean_text'].values
y_intent = df_big['intent'].values

# Train-test split
X_train_int, X_test_int, y_train_int, y_test_int = train_test_split(
    X_intent, y_intent, test_size=0.2, random_state=RANDOM_STATE, stratify=y_intent
)

# TF-IDF vectorization
tfidf_int = TfidfVectorizer(ngram_range=(1,2), min_df=1, max_df=0.95)
X_train_int_tfidf = tfidf_int.fit_transform(X_train_int)
X_test_int_tfidf = tfidf_int.transform(X_test_int)

# Logistic Regression classifier
clf_int = LogisticRegression(max_iter=300, random_state=RANDOM_STATE)
clf_int.fit(X_train_int_tfidf, y_train_int)

# Evaluate
y_pred_int = clf_int.predict(X_test_int_tfidf)
acc_int = accuracy_score(y_test_int, y_pred_int)
print(f"Intent Classification Accuracy: {acc_int:.4f}")
print(classification_report(y_test_int, y_pred_int, digits=4))

# Save model and vectorizer
joblib.dump(tfidf_int, 'tfidf_int.joblib')
joblib.dump(clf_int, 'clf_int.joblib')

#Explanation:
#Trains a robust intent classifier on synthetic dataset.
#Saves vectorizer and model for deployment.

Intent Classification Accuracy: 1.0000
              precision    recall  f1-score   support

   complaint     1.0000    1.0000    1.0000        60
     goodbye     1.0000    1.0000    1.0000        60
    greeting     1.0000    1.0000    1.0000        60
        info     1.0000    1.0000    1.0000        60
      praise     1.0000    1.0000    1.0000        60
    question     1.0000    1.0000    1.0000        60

    accuracy                         1.0000       360
   macro avg     1.0000    1.0000    1.0000       360
weighted avg     1.0000    1.0000    1.0000       360



['clf_int.joblib']

In [ ]:
# Cell 9: HuggingFace Emotion Model

# Load pre-trained emotion detection model
emotion_model_name = "SamLowe/roberta-base-go_emotions"
tokenizer_emotion = AutoTokenizer.from_pretrained(emotion_model_name)
model_emotion = AutoModelForSequenceClassification.from_pretrained(emotion_model_name)
id2label_emotion = model_emotion.config.id2label

def predict_emotion(text):
    """
    Predict top emotion and probability scores for all emotions.
    """
    if not text.strip():
        return "No text provided", {}
    inputs = tokenizer_emotion(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model_emotion(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
    top_idx = int(np.argmax(probs))
    top_emotion = id2label_emotion[top_idx]
    scores = {id2label_emotion[i]: float(probs[i]) for i in range(len(probs))}
    return top_emotion, scores


#Explanation:
#Uses GoEmotions model from HuggingFace.
#Returns top emotion and probabilities for all emotions.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Cell 10: HuggingFace Toxicity Detection

toxicity_model_name = "unitary/toxic-bert"
tokenizer_toxic = AutoTokenizer.from_pretrained(toxicity_model_name)
model_toxic = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name)
tox_labels = ['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_hate']

def predict_toxicity(text):
    """
    Predicts probability for toxicity labels.
    """
    if not text.strip():
        return {}
    inputs = tokenizer_toxic(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model_toxic(**inputs)
    probs = torch.sigmoid(outputs.logits)[0].numpy()
    return {tox_labels[i]: float(probs[i]) for i in range(len(tox_labels))}

#Explanation:
#Multi-label classification for toxicity detection.
#Uses sigmoid because a text can have multiple toxicity labels.


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Cell 11: Named Entity Recognition (NER) using SpaCy

nlp_ner = spacy.load("en_core_web_sm")

def extract_entities(text):
    """
    Extracts named entities from text (PERSON, ORG, GPE, etc.)
    """
    if not text.strip():
        return {}
    doc = nlp_ner(text)
    entities = {}
    for ent in doc.ents:
        entities.setdefault(ent.label_, []).append(ent.text)
    return entities

#Explanation:
#Extracts named entities using SpaCy small model.
#Returns dictionary {entity_type: [entities]}.

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import spacy
import subprocess
import sys

# ---------------------------
# Installation Checks
# ---------------------------
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import spacy_transformers
except ImportError:
    install_package("spacy[transformers]")

# ---------------------------
# Load SpaCy NER Model
# ---------------------------
try:
    nlp_ner = spacy.load("en_core_web_trf")
except Exception as e:
    print(f"Warning: en_core_web_trf failed ({e}), falling back to en_core_web_sm.")
    try:
        nlp_ner = spacy.load("en_core_web_sm")
    except OSError:
        install_package("en_core_web_sm")
        nlp_ner = spacy.load("en_core_web_sm")

# ---------------------------
# Load Models
# ---------------------------
sentiment_model = pipeline("sentiment-analysis")
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_name)
emotion_id2label = emotion_model.config.id2label

toxicity_model_name = "unitary/toxic-bert"
tox_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name)
tox_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name)

# ---------------------------
# Helper Functions
# ---------------------------
def analyze_sentiment(text):
    result = sentiment_model(text)[0]
    label = result['label']
    color = "green" if label.lower() == "positive" else "red"
    return label.capitalize(), color

def analyze_emotion(text, threshold=0.1):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = emotion_model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()
    emotions = []
    for idx, prob in enumerate(probs):
        if prob >= threshold:
            emotion = emotion_id2label[idx]
            if emotion.lower() in ["joy", "love", "relief", "admiration", "optimism", "excited", "surprise"]:
                color = "green"
            elif emotion.lower() in ["neutral", "curiosity"]:
                color = "gray"
            else:
                color = "red"
            emotions.append((emotion.capitalize(), color))
    return emotions if emotions else [("Neutral", "gray")]

def analyze_toxicity(text):
    inputs = tox_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = tox_model(**inputs)
    probs = torch.sigmoid(outputs.logits)[0].cpu().numpy()
    score = max(probs)
    if score < 0.2:
        return "Low", "green"
    elif score < 0.5:
        return "Moderate", "orange"
    else:
        return "High", "red"

def extract_entities(text):
    doc = nlp_ner(text)
    entities = {}
    for ent in doc.ents:
        entities.setdefault(ent.label_, []).append(ent.text)
    keywords = {"PRODUCT": ["phone", "laptop", "computer", "tablet", "watch"]}
    for label, words in keywords.items():
        for word in words:
            if word.lower() in text.lower():
                entities.setdefault(label, []).append(word)
    return entities if entities else None

def detect_intent(text):
    text_lower = text.lower()

    greetings = ["hi", "hello", "hey", "good morning", "good evening"]
    questions = ["who", "what", "where", "when", "why", "how"]
    complaints = ["not working", "problem", "issue", "tired", "can't", "cannot",
                  "error", "fail", "crash", "frustrated", "hate", "annoyed",
                  "nothing is going right", "not going right", "upset", "sad"]

    # Complaint (strongest)
    if any(word in text_lower for word in complaints):
        return "Complaint"

    # Question
    if any(text_lower.startswith(q) or q in text_lower for q in questions) or "?" in text_lower:
        return "Question"

    # Greeting
    if any(text_lower.strip().startswith(word) for word in greetings):
        return "Greeting"

    return "Statement"

def mental_health_insight(text, emotions):
    mapping_keywords = {
        "Disappointment / Sadness": ["disappointed", "sad", "frustrated", "unhappy", "hate", "annoyed"],
        "Excitement / Happiness": ["excited", "happy", "joy", "thrilled", "optimistic", "love", "admiration"],
        "Anxiety / Stress": ["anxious", "stressed", "worried", "nervous", "tense", "overwhelmed"],
        "Depression": ["depressed", "hopeless", "lonely", "worthless", "gloomy"],
        "Phobia / Fear": ["afraid", "fear", "scared", "panic", "phobia"],
        "OCD / Compulsive": ["repetitive", "check", "control", "ritual", "compulsion"]
    }
    colors = {
        "Disappointment / Sadness": "red",
        "Excitement / Happiness": "green",
        "Anxiety / Stress": "orange",
        "Depression": "darkred",
        "Phobia / Fear": "purple",
        "OCD / Compulsive": "blue"
    }
    insights = []
    text_lower = text.lower()
    for label, keywords in mapping_keywords.items():
        if any(w in text_lower for w in keywords):
            insights.append((label, colors[label]))
    for emo, _ in emotions:
        emo_lower = emo.lower()
        if emo_lower in ["anger", "sadness", "disgust", "fear", "disappointment"]:
            insights.append(("Disappointment / Sadness", colors["Disappointment / Sadness"]))
        elif emo_lower in ["joy", "love", "relief", "optimism", "admiration", "excited", "surprise"]:
            insights.append(("Excitement / Happiness", colors["Excitement / Happiness"]))
        elif emo_lower in ["anxiety", "stress", "worry", "nervous"]:
            insights.append(("Anxiety / Stress", colors["Anxiety / Stress"]))
        elif emo_lower in ["depression", "hopeless", "lonely", "gloomy"]:
            insights.append(("Depression", colors["Depression"]))
        elif emo_lower in ["fear", "phobia", "panic", "scared"]:
            insights.append(("Phobia / Fear", colors["Phobia / Fear"]))
    # Remove duplicates
    seen = set()
    unique_insights = []
    for label, color in insights:
        if label not in seen:
            unique_insights.append((label, color))
            seen.add(label)
    return unique_insights if unique_insights else None

def display_entities_with_badges(entities):
    if not entities:
        return "None"
    color_map = {"PERSON": "blue","ORG": "purple","GPE": "teal","LOC": "teal","PRODUCT": "green",
                 "DATE": "orange","TIME": "orange","MISC": "gray"}
    badges = []
    for label, values in entities.items():
        color = color_map.get(label, "gray")
        for v in values:
            badges.append(f"<span style='color:{color}; font-weight:bold'>{v} ({label})</span>")
    return " ".join(badges)

def display_emotions_with_badges(emotions):
    return " ".join([f"<span style='color:{color}; font-weight:bold'>{label}</span>" for label, color in emotions])

# ---------------------------
# NEW: Emotional Safety Detection
# ---------------------------
def detect_emotional_safety(text):
    unsafe_keywords = ["suicide", "kill myself", "end it all", "hopeless", "worthless", "panic", "scared", "fear", "alone"]
    text_lower = text.lower()
    if any(word in text_lower for word in unsafe_keywords):
        return " Emotional Risk Detected"
    return " Seems Emotionally Safe"

# ---------------------------
# NEW: Supportive Response Generator (with emotional opening detection)
# ---------------------------
def supportive_response(text):
    text_lower = text.lower()

    # Emotional opening / sharing
    opening_keywords = ["i feel", "nothing in my life", "i just", "i'm feeling", "i am feeling"]
    if any(kw in text_lower for kw in opening_keywords):
        return "I'm here for you. Sharing how you feel is brave, and it's okay to feel this way. You're not alone."

    # Sad / depressed
    if any(word in text_lower for word in ["sad", "depressed", "hopeless", "worthless", "lonely"]):
        return "I'm here for you. It's okay to feel this way. Take a deep breath and remember you're not alone."

    # Anxious / worried
    elif any(word in text_lower for word in ["anxious", "worried", "nervous", "panic"]):
        return "I understand you're feeling anxious. Try focusing on your breathing and take things one step at a time."

    # Angry / frustrated
    elif any(word in text_lower for word in ["angry", "frustrated", "annoyed"]):
        return "It's natural to feel frustrated. Take a moment to relax and let out your feelings safely."

    else:
        return "Thank you for sharing. Remember, it's okay to express your emotions. You're heard."

# ---------------------------
# Streamlit UI
# ---------------------------
st.set_page_config(page_title="Insightify", layout="centered")
st.title("Insightify")
st.write("Analyze **Sentiment, Intent, Emotion, Named Entities, Toxicity, and Mental Health Insights**")

user_input = st.text_area("Enter text here:")

serious_toggle = st.checkbox("Show only serious mental health concerns (Depression, Phobia, OCD)")

if st.button("Analyze") and user_input.strip():
    sentiment, sentiment_color = analyze_sentiment(user_input)
    st.markdown(f"**Sentiment:** <span style='color:{sentiment_color}'>{sentiment}</span>", unsafe_allow_html=True)

    intent = detect_intent(user_input)
    st.markdown(f"**Intent:** {intent}")

    emotions = analyze_emotion(user_input)
    st.markdown("**Emotion:**")
    st.markdown(display_emotions_with_badges(emotions), unsafe_allow_html=True)

    entities = extract_entities(user_input)
    st.markdown("**Named Entities:**")
    st.markdown(display_entities_with_badges(entities), unsafe_allow_html=True)

    tox_level, tox_color = analyze_toxicity(user_input)
    st.markdown(f"**Toxicity:** <span style='color:{tox_color}'>{tox_level}</span>", unsafe_allow_html=True)

    insights = mental_health_insight(user_input, emotions)
    st.markdown("**Mental Health Insight:**")
    if insights:
        if serious_toggle:
            serious_insights = [i for i in insights if i[0] in ["Depression","Phobia / Fear","OCD / Compulsive"]]
            badges = " ".join([f"<span style='color:{color}; font-weight:bold'>{label}</span>" for label, color in serious_insights])
            st.markdown(badges if badges else "None", unsafe_allow_html=True)
        else:
            badges = " ".join([f"<span style='color:{color}; font-weight:bold'>{label}</span>" for label, color in insights])
            st.markdown(badges, unsafe_allow_html=True)
    else:
        st.write("None")

    st.markdown("**Emotional Safety Detection:**")
    st.markdown(detect_emotional_safety(user_input))

    st.markdown("**Supportive Response:**")
    st.markdown(supportive_response(user_input))






Writing app.py


In [ ]:
# Install required packages (run once)
!pip install streamlit pyngrok --quiet

from pyngrok import ngrok
import subprocess
import time
import requests

# Authenticate ngrok
NGROK_AUTHTOKEN = "38WffZksOZDdF7tNQFkslCLVS3w_6AQsBgZKfsvxFfso4A6ZU"
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# Kill old tunnels / processes
ngrok.kill()
subprocess.run("pkill streamlit", shell=True)
subprocess.run("fuser -k 8501/tcp", shell=True)

# Start Streamlit app
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for Streamlit to start (poll localhost)
max_wait = 60  # seconds
start_time = time.time()
while True:
    try:
        r = requests.get("http://localhost:8501")
        if r.status_code == 200:
            break
    except:
        pass
    if time.time() - start_time > max_wait:
        print("Streamlit did not start in time. Check app.py for errors.")
        break
    time.sleep(1)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url.public_url)


Your Streamlit app is live at: https://willetta-postorbital-amberly.ngrok-free.dev
